---
title: Performance Tips
engine: julia
aliases:
 - ../../tutorials/docs-13-using-turing-performance-tips/index.html
---

```{julia}
#| echo: false
#| output: false
using Pkg;
Pkg.instantiate();
```

This section briefly summarises a few common techniques to ensure good performance when using Turing.
We refer to [the Julia documentation](https://docs.julialang.org/en/v1/manual/performance-tips/index.html) for general techniques to ensure good performance of Julia programs.

## Use multivariate distributions

It is generally preferable to use multivariate distributions if possible.

The following example:

```{julia}
using Turing
@model function gmodel(x)
    m ~ Normal()
    for i in eachindex(x)
        x[i] ~ Normal(m, 0.2)
    end
end
```

can be directly expressed more efficiently using a simple transformation:

```{julia}
using FillArrays

@model function gmodel(x)
    m ~ Normal()
    return x ~ MvNormal(Fill(m, length(x)), 0.04 * I)
end
```

## Choose your AD backend

Automatic differentiation (AD) makes it possible to use modern, efficient gradient-based samplers like NUTS and HMC.
This, however, also means that using a performant AD system is incredibly important.
Turing currently supports several AD backends, including [ForwardDiff](https://github.com/JuliaDiff/ForwardDiff.jl) (the default), [Mooncake](https://github.com/chalk-lab/Mooncake.jl), and [ReverseDiff](https://github.com/JuliaDiff/ReverseDiff.jl).

For many common types of models, the default ForwardDiff backend performs well, and there is no need to worry about changing it.
However, if you need more speed, you can try different backends via the standard [ADTypes](https://github.com/SciML/ADTypes.jl) interface by passing an `AbstractADType` to the sampler with the optional `adtype` argument, e.g. `NUTS(; adtype = AutoMooncake())`.

Generally, `adtype = AutoForwardDiff()` is likely to be the fastest and most reliable for models with few parameters (say, less than 20 or so), while reverse-mode backends such as `AutoMooncake()` or `AutoReverseDiff()` will perform better for models with many parameters or linear algebra operations.
If in doubt, you can benchmark your model with different backends to see which one performs best.
See the [Automatic Differentiation]({{<meta usage-automatic-differentiation>}}) page for details.

### Special care for ReverseDiff with a compiled tape

For large models, the fastest option is often ReverseDiff with a compiled tape, specified as `adtype=AutoReverseDiff(; compile=true)`.
However, it is important to note that if your model contains any branching code, such as `if`-`else` statements, **the gradients from a compiled tape may be inaccurate, leading to erroneous results**.
If you use this option for the (considerable) speedup it can provide, make sure to check your code for branching and ensure that it does not affect the gradients.
It is also a good idea to verify your gradients with another backend.

## Ensure that types in your model can be inferred

For efficient gradient-based inference, e.g. using HMC, NUTS or ADVI, it is important to ensure the types in your model can be inferred.

The following example with abstract types

```{julia}
@model function tmodel(x, y)
    p, n = size(x)
    params = Vector{Real}(undef, n)
    for i in 1:n
        params[i] ~ truncated(Normal(); lower=0)
    end

    a = x * params
    return y ~ MvNormal(a, I)
end
```

can be transformed into the following representation with concrete types:

```{julia}
@model function tmodel(x, y, ::Type{T}=Float64) where {T}
    p, n = size(x)
    params = Vector{T}(undef, n)
    for i in 1:n
        params[i] ~ truncated(Normal(); lower=0)
    end

    a = x * params
    return y ~ MvNormal(a, I)
end
```

Alternatively, you could use `filldist` in this example:

```{julia}
@model function tmodel(x, y)
    params ~ filldist(truncated(Normal(); lower=0), size(x, 2))
    a = x * params
    return y ~ MvNormal(a, I)
end
```

You can use DynamicPPL's debugging utilities to find types in your model definition that the compiler cannot infer.
These will be marked in red in the Julia REPL (much like when using the `@code_warntype` macro).

For example, consider the following model:

```{julia}
@model function tmodel(x)
    p = Vector{Real}(undef, 1)
    p[1] ~ Normal()
    p = p .+ 1
    return x ~ Normal(p[1])
end
```

Because the element type of `p` is an abstract type (`Real`), the compiler cannot infer a concrete type for `p[1]`.
To detect this, we can use

```{julia}
#| eval: false
model = tmodel(1.0)

using DynamicPPL
DynamicPPL.DebugUtils.model_warntype(model)
```

In this particular model, the following call to `getindex` should be highlighted in red (the exact numbers may vary):

```
[...]
│    %120 = p::AbstractVector
│    %121 = Base.getindex(%120, 1)::Any
[...]
```